### Project imports

In [1]:
import requests
import json
from pprint import pprint
import numpy as np
from datetime import date, timedelta
import urllib3
urllib3.disable_warnings()

## Declare global variables

In [2]:
START_DATE = '2012-01-02'
END_DATE = '2017-06-30'
user = 'epita_user_4'
pwd = 'dolphin21903'
RISK_FREE = 5e-4
PORTFOLIO_VALUE = 10000000
server_url = 'https://dolphin.jump-technology.com:3389/api/v1/'
ratio_list = [15, 17, 18, 19, 20, 21, 22, 29]
ratio_identifiers_dict = {
    'beta' : 15, 'annual_return' : 17, 'volatility' : 18, 'correlation' : 19,
    'sharpe' : 20, 'global_return' : 21,'value_at_risk' : 22, 'action_composition': 29
}

## Server setup

In [3]:
class Server:
    def __init__(self, server_url, user, pwd):
        self.server_url = server_url
        self.user = user
        self.pwd = pwd
        self.session = requests.Session()
        self.session.auth = (user, pwd)

    def get(self, req_str):
        data = None
        try:
            url = server_url + req_str
            response = self.session.get(url, verify=False)
            data = json.loads(response.text)
        except Exception as e:
            print('Exception in Server Get : ', str(e))
        return data
    
    def post(self, req_str, obj):
        """
        req_str : the request string
        obj : the object to post
        """
        data = None
        try:
            url = server_url + req_str
            response = self.session.post(url=url, json=obj, verify=False)
            data = json.loads(response.text)
        except Exception as e:
            print('Exception : ', str(e))
        return data
    
    def put(self, req_str, obj):
        url = self.server_url + req_str
        data = self.session.put(url, data=obj, verify=False)
        return data
    
server = Server(server_url, user, pwd)

## Collect needed data

In [4]:
class ApiManager:
    def __init__(self, server, ratio_list, ratio_identifiers_dict, risk_free,
                 start_date, end_date, portfolio_value):
        self.server = server
        self.RATIO_LIST = ratio_list
        self.ratio_identifiers_dict = ratio_identifiers_dict
        self.RISK_FREE = risk_free
        self.START_DATE = start_date
        self.END_DATE = end_date
        self.PORTFOLIO_VALUE = portfolio_value
        
        ################################## ASSET IDS ###########################################
        
        self.assets, self.asset_ids = self.__get_assets()
        self.ids_idx_db, self.ids_db_idx = self.__create_asset_id_mapping()
        
        ################################## RATIO LIST ###########################################
        
        self.assets_ratio_list = self.__get_assets_ratio_list(self.START_DATE, self.END_DATE) 
        ratio_ids = [ratio_identifiers_dict[x] for x in ['global_return', 'sharpe', 'volatility']]
        self.global_returns, self.sharpes, self.volatilities = [x for x in self.__get_ratio(ratio_ids)]

        ################################## DAILY RETURN ###########################################

        self.assets_daily_returns = self.__get_assets_daily_returns(self.START_DATE, self.END_DATE)
        
        ################################## PORTFOLIO ###########################################
        
        self.portfolio = self.__get_portfolio()
        
        
    ################################## ASSET IDS ###########################################

    def __get_assets(self, only_euro=True):
        #assets = server.get('asset?columns=ASSET_DATABASE_ID&columns=LABEL')
        assets = self.server.get('asset')
        asset_ids = []
        if only_euro:
            assets_eur = []
            for asset in assets:
                if asset['CURRENCY']['value'] == 'EUR':
                    assets_eur.append(asset)
                    asset_ids.append(asset["ASSET_DATABASE_ID"]["value"])
            assets = assets_eur
        else: asset_ids = [asset["ASSET_DATABASE_ID"]["value"] for asset in assets]
        return assets, asset_ids
    
    def __create_asset_id_mapping(self):
        """Create the mapping between the assets api ids and the index"""
        ids_idx_db = {}; ids_db_idx = {}
        for idx, id_asset in enumerate(self.asset_ids):
            ids_idx_db[idx] = id_asset
            ids_db_idx[id_asset] = idx
            
        return ids_idx_db, ids_db_idx
    
    def get_db_ids(self, idx_ids):
        return [self.ids_idx_db[idx_id] for idx_id in idx_ids]

    def get_idx_ids(self, db_ids):
        ids_vector = np.zeros((len(db_ids)))
        for idx, db_id in enumerate(db_ids):
            ids_vector[idx] = self.ids_db_idx[db_id]
        return ids_vector
    
    ################################## RATIO LIST ###########################################

    def __get_assets_ratio_list(self, start_date, end_date):
        obj = {
            'ratio':self.RATIO_LIST,
            'asset':self.asset_ids,
            'bench':None,
            'startDate':start_date,
            'endDate':end_date,
            'frequency':None
        }
        return self.server.post('ratio/invoke', obj)

    def __get_ratio(self, ratio_ids):
        """
        Note: to know the id of the asset value in ith index, use the mapping dict
        assets_id_dict_idx[i] -> asset_id
        """
        ratio_values = []
        for ratio_id in ratio_ids:
            aux = []
            for asset_id in self.asset_ids:
                aux.append(self.__str_to_float(self.assets_ratio_list[asset_id][str(ratio_id)]['value']))
            ratio_values.append(np.array(aux))
        return ratio_values
    
    def __str_to_float(self, str_value):
        return float(str_value.replace(',', '.'))
    
    ################################## DAILY RETURN ###########################################
    
    def __get_assets_daily_returns(self, start_date, end_date):
        daily_returns = self.__get_daily_returns(start_date, end_date)
        sorted_dates = self.__get_return_dates(daily_returns, start_date, end_date)
        assets_returns = np.full((len(sorted_dates), len(self.asset_ids)), None)
        for i, asset_id in enumerate(self.asset_ids):
            returns = daily_returns[asset_id]
            for r in returns:
                date_idx = sorted_dates.index(r['date'])
                assets_returns[date_idx,i] = r['return']
        assets_returns = self.__none_values_manager(assets_returns, sorted_dates)
        return assets_returns
    
    def __none_values_manager(self, assets_returns, sorted_dates):
        # Remove none values and replace them with the mean of values before and after
        none_idx = np.where(assets_returns == None)

        for i, j in zip(none_idx[0], none_idx[1]):
            if i in range(1, len(sorted_dates) - 1) and assets_returns[i-1,j] and assets_returns[i+1,j]:
                assets_returns[i,j] = (assets_returns[i-1,j] + assets_returns[i+1,j]) / 2
            elif i == 0 and assets_returns[i+1,j]:
                assets_returns[i,j] = assets_returns[i+1,j]
            elif i == len(sorted_dates) - 1 and assets_returns[i-1,j]:
                assets_returns[i,j] = assets_returns[i-1,j]
            else:
                assets_returns[i,j] = 0
        return assets_returns
    
    def __get_daily_returns(self, start_date, end_date):
        assets_returns = {}
        for asset_id in self.asset_ids:
            returns_obj = self.server.get(
                ('asset/{}/quote?start_date={}&end_date={}'.format(asset_id, start_date, end_date)))
            assets_returns[asset_id] = [r for r in returns_obj]
            print(asset_id)
        return assets_returns
    
    def __get_return_dates(self, daily_returns, start_date, end_date):
        """Returns all the dates in the period sorted in the increasing order"""
        dates = {start_date, end_date}
        for asset_id in self.asset_ids:
            returns = daily_returns[asset_id]
            for r in returns:
                dates.add(r['date'])
        return sorted(dates, key=lambda d: tuple(map(int, d.split('-'))))


    ################################## PORTFOLIO ###########################################
    
    def __get_portfolio(self):
        request = 'asset?columns=ASSET_DATABASE_ID&columns=LABEL&columns=TYPE&TYPE=PORTFOLIO'
        portfolio_infos = self.server.get(request)[0]
        portfolio = {
            'id' : portfolio_infos['ASSET_DATABASE_ID']['value'],
            'label' : portfolio_infos['LABEL']['value']
        }
        return portfolio
    
    def post_portfolio(self, asset_db_ids, asset_weights, start_date, end_date):
        assets_dict = self.__get_asset_quantities(
            asset_db_ids, asset_weights, start_date, end_date)
        potfolio_dict = {
            'currency': {'code': 'EUR'},
            'label': self.portfolio['label'],
            'type': 'front',
            'values': {start_date: assets_dict}}
        portfolio_obj = json.dumps(potfolio_dict)
        data = server.put("/portfolio/{}/dyn_amount_compo".format(self.portfolio['id']), portfolio_obj)
        return data
    
    def __get_asset_quantities(self, asset_db_ids, asset_weights, start_date, end_date):
        assets_dict = []
        for idx, asset_id in enumerate(asset_db_ids):
            asset_infos = server.get('asset/{}/quote?start_date={}&end_date={}'.format(
                asset_id, start_date, end_date))
            closure_value = asset_infos[-1]['close']
            quantity = int(asset_weights[idx] * (self.PORTFOLIO_VALUE / closure_value))
            assets_dict.append({'asset': {'asset': int(asset_id), 'quantity': int(quantity)}})
        return assets_dict

In [5]:
api_manager = ApiManager(server, ratio_list, ratio_identifiers_dict,
                         RISK_FREE, START_DATE, END_DATE, PORTFOLIO_VALUE)

405
61
67
144
376
109
31
68
169
410
494
489
508
471
483
481
482
371
147
384
341
349
346
368
386
347
490
442
530
168
356
202
110
233
54
408
528
537
28
441
466
411
422
416
452
173
511
225
6
207
545
544
549
103
262
9
391
364
208
29
390
531
467
498
532
409
526
470
26
440
117
426
423
252
342
497
488
533
461
123
66
403
529
541
348
425
379
527
462
53
72
182
65
308
140
331
170
30
7
82
493
534
509
552
413
457
515
517
357
427
314
535
484
383
374
353
460
373
14
179
106
13
97
235
185
512
392
477
367
351
500
419
514
525
429
501
475
445
378
359
188
226
113
380
45
538
507
557
437
152
4
339
69
505
292
421
212
213
122
336
553
540
322
449
463
393
412
343
394
465
102
145
200
64
448
447
518
118
81
496
166
139
404
453
516
443
439
476
381
176
27
473
417
556
521
112
396
406
414
377
93
492
451
450
319
19
399
395
499
402
520
34
444
32
352
546
370
570
558
33
519
263
190
547
433
480
478
479
247
175
146
25
485
10
238
306
189
420
522
389
455
548
183
366
365
198
11
167
217
104
536
486
282
193
550
358
446
320
22
291

## Problem optimization

In [6]:
from cvxopt import matrix, solvers

class PotfolioOptimizer:
    def __init__(self, api_manager):
        self.api_manager = api_manager
    
    ################################### COVARIANCE MATRIX ################################
    def __get_covariance_matrix(self, asset_idx_ids):
        daily_returns = self.api_manager.assets_daily_returns
        daily_return_target = np.zeros((len(asset_idx_ids), daily_returns.shape[0]))
        for idx, asset_id in enumerate(asset_idx_ids):
            daily_return_target[idx] = daily_returns[:,idx]
        # TODO: test this one
        #daily_return_target = [daily_returns[:,idx] for idx in assets_ids_target]
        return np.cov(daily_return_target)

    ################################## GLOBAL RETURNS ####################################
    def __get_global_returns(self, asset_idx_ids):
        ids_int = [int(x) for x in asset_idx_ids]
        return self.api_manager.global_returns[ids_int]
    
    #################################### OPTIMIZER #######################################

    def optimize_portfolio_weights(self, asset_idx_ids, weight_min=0.01, weight_max=0.1):
        """The portfolio composition is fixed (the assets contained in th portfolio),
        the goal is to find the best weights for these assets that maximizes the Sharpe
        Returns the optimized weights and the protfolio_sharpe
        """
        global_returns = self.__get_global_returns(asset_idx_ids)
        covariance_matrix = self.__get_covariance_matrix(asset_idx_ids)
        weights = self.optimizer(global_returns, covariance_matrix, weight_min, weight_max)
        portfolio_sharpe = self.evaluate_portfolio(weights, global_returns, covariance_matrix)
        db_ids = self.api_manager.get_db_ids(asset_idx_ids)
        return db_ids, weights, portfolio_sharpe

    def optimize_portfolio_composition(self, asset_db_ids, min_weight=0.001, max_weight=0.05):
        idx_ids = self.api_manager.get_idx_ids(asset_db_ids)
        db_ids, weights, portfolio_sharpe = self.optimize_portfolio_weights(idx_ids, min_weight, max_weight)
        idx_max_weights = weights.argsort()[-20:][::-1]
        db_ids, weights, portfolio_sharpe = self.optimize_portfolio_weights(idx_max_weights)
        return db_ids, weights, portfolio_sharpe
    
    # TODO: test with different values for alpha
    def optimizer(self, global_returns, covariance_matrix, weight_min, weight_max, risk_tolerance=1):
        nb = covariance_matrix.shape[0]
        # Problem definition
        P = matrix(2 * covariance_matrix, tc='d')
        q = matrix(-risk_tolerance * global_returns, tc='d')
        #G = matrix(-np.identity(nb), tc='d')
        #h = matrix(np.zeros(nb))
        A = matrix(np.ones((nb))).T
        b = matrix(np.ones((1)))
        G = matrix(np.vstack((-np.identity(nb), np.identity(nb))), tc='d')
        h = matrix(np.hstack((-np.full((nb), weight_min), np.full((nb), weight_max))), tc='d')
        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b)['x']
        return np.asarray(sol).reshape((nb,))
    
    ############################### PORTFOLIO EVALUATION ##################################

    def evaluate_portfolio(self, weights, global_returns, cov, is_annuel=True):
        volatility_portfolio = np.sqrt(weights.T @ cov @ weights)
        if is_annuel:
            return_portfolio = (np.power(1 + global_returns.T @ weights, 365 / 1642) - 1)
        else:
            return_portfolio = global_returns.T @ weights
        sharpe_portfolio = (return_portfolio - RISK_FREE) / volatility_portfolio
        return sharpe_portfolio

In [7]:
portfolio_optimizer = PotfolioOptimizer(api_manager)

### Our solution: (Markowitz)

In [14]:
db_ids, weights, portfolio_sharpe = portfolio_optimizer.optimize_portfolio_composition(api_manager.asset_ids)
print("sharpe = ", portfolio_sharpe)
api_manager.post_portfolio(db_ids, weights, START_DATE, END_DATE)

sharpe =  45.0740854896


<Response [404]>

### Assets with best Sharpe
- __Weights = 0.05__

In [9]:
best_sharpes_idx_id = np.argsort(api_manager.sharpes)[-20:][::-1]
db_ids, weights, portfolio_sharpe = portfolio_optimizer.optimize_portfolio_weights(
    best_sharpes_idx_id, weight_min=0.05, weight_max=0.05)
print("sharpe = ", portfolio_sharpe)

sharpe =  19.3160755231


- __Min weight = 0.01, Max weight = 0.1__

In [10]:
db_ids, weights, portfolio_sharpe = portfolio_optimizer.optimize_portfolio_weights(best_sharpes_idx_id)
print("sharpe = ", portfolio_sharpe)

sharpe =  29.1368363244


### Random assets
- __Weights = 0.05__

In [11]:
import random
perfs = []
for i in range(200):
    ids = random.sample(range(0, len(api_manager.asset_ids)), 20)
    db_ids, weights, portfolio_sharpe = portfolio_optimizer.optimize_portfolio_weights(
        best_sharpes_idx_id, weight_min=0.05, weight_max=0.05)
    perfs.append(portfolio_sharpe)
    
perfs = np.array(perfs)
print("sharpe = ", np.amax(perfs))

sharpe =  19.3160755231


- __Min weight = 0.01, Max weight = 0.1__

In [12]:
import random
perfs = []
for i in range(200):
    ids = random.sample(range(0, len(api_manager.asset_ids)), 20)
    db_ids, weights, portfolio_sharpe = portfolio_optimizer.optimize_portfolio_weights(best_sharpes_idx_id)
    perfs.append(portfolio_sharpe)
    
perfs = np.array(perfs)
print("sharpe = ", np.amax(perfs))

sharpe =  29.1368363244
